In [ ]:
import deeplabcut as dlc
import cv2
import numpy as np
import os 
import sys # Para poder usar sys.exit()
import yaml # Para leer el archivo de configuración
import matplotlib.pyplot as plt
print('Fin de la importación de módulos')

In [ ]:
# Ruta al archivo de configuración de SLEAP
path_config_file = 'Experimento Olfato-JJ-2024-02-19/config2.yaml'

# Verificamos que la configuración exista antes de continuar
if not os.path.exists(path_config_file):
    print(f'No existe el archivo de configuración en {path_config_file}')
else:
    print(f'Cargando archivo {path_config_file}')
    # Leo el archivo de configuración
    with open(path_config_file) as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
    print(config)

    # Verificamos el 'project_path' porque es esencial que este configurado con la ubicación absoluta real
    if not os.path.exists(config['project_path']):
        print(f'No existe el directorio {config["project_path"]} configurado en {path_config_file}')
    else:
        print(f'El directorio {config["project_path"]} existe')


In [ ]:
video_path = 'experiments/A-7.mp4'

if not os.path.exists(video_path):
    print(f'No existe el archivo de video en {video_path}')
    # y detenemos la ejecución de la celda con
    raise SystemExit("Stop right there!")
else:
    print(f'Cargando archivo {video_path}')

# se puede analizar el video en python con opencv. Las propiedades del video pueden ser obtenidas con el siguiente código
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
print(ret)
print(cap.get(cv2.CAP_PROP_FPS))
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(cap.get(cv2.CAP_PROP_FOURCC))

random_frame = np.random.randint(0, cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Se puede obtener un frame en particular con el siguiente código, y se puede mostrar con matplotlib
cap.set(cv2.CAP_PROP_POS_FRAMES, random_frame)
ret, frame = cap.read()

fig, ax = plt.subplots()
plt.imshow(frame, cmap='viridis')
plt.axis('off')
plt.title(f'Frame {random_frame}')
plt.show()

cap.release()


In [ ]:
import cv2

# Definir la ROI (x, y, ancho, alto)
roi_x, roi_y, roi_w, roi_h = 30, 70, 520, 245

video_path = 'experiments/A-7.mp4'
video_out = 'experiments/A-7_crop.mp4'

# Abrir el video
cap = cv2.VideoCapture(video_path)

# Obtener propiedades del video
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Definir el codec y crear el objeto VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_out, fourcc, fps, (roi_w, roi_h))

# Procesar cada frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Recortar el frame a la ROI
    roi_frame = frame[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]
    
    # Escribir el frame recortado en el nuevo video
    out.write(roi_frame)

# Liberar los objetos
cap.release()
out.release()


In [ ]:
import time
# ahora podemos usar este video para analizarlo con deeplabcut rapidamente y ver como se comporta la inferencia en un video corto

video_anal = 'experiments/A-7_crop.mp4'

start_time = time.time()
dlc.analyze_videos(path_config_file, 
                   [video_anal], 
                          videotype='mp4', 
                          shuffle=1, 
                          trainingsetindex=0, 
                          gputouse=None, 
                          save_as_csv=True, 
)

print(f'El análisis de {video_anal} tardó {time.time()-start_time} segundos')

In [ ]:
dlc.filterpredictions(path_config_file,
                    [video_anal], 
                    shuffle=1, 
                    trainingsetindex=0, 
                    filtertype='arima',  
                    windowlength=5, 
                    p_bound=0.7, ARdegree=3, MAdegree=1, alpha=0.01, 
                    save_as_csv=True, destfolder=None, 
                    modelprefix='', track_method='', 
                    return_data=False)

In [ ]:
output_video = '/home/juan/Dropbox/Documentos/Programacion/Ratones Lorena Rela/experiments/C-3_anotado.mp4'

dlc.create_labeled_video(path_config_file, 
                            [output_video], 
                            shuffle=1, 
                            trainingsetindex=0, 
                            filtered=False, 
                            save_frames=False, 
 

                            displayedbodyparts='all', 
 
                            draw_skeleton=False, 
                            trailpoints=0, 


                            videotype='mp4', 

)